In [1]:
from Speaker_identification import Speaker_identification
from density_func import density_func, naive_G_U
from sklearn.mixture import GaussianMixture
import seaborn as sn
import pandas as pd
import matplotlib.pyplot as plt
#default

import pickle
import numpy as np
import time

#loadmat
from process_bar import ShowProcess
from scipy.io import loadmat,savemat

'''
Import features
'''

#save_path="D:\\LAB\\lab\\task_2_version_4\\features.txt"
save_path="/Users/Mata/Documents/lab/task_2_version_4/features.txt"
f = open(save_path,'rb')
features=pickle.load(f)
f.close()

#save_path="D:\\LAB\\lab\\task_2_version_4\\unknown_features.txt"
save_path="/Users/Mata/Documents/lab/task_2_version_4/unknown_feature.txt"
f = open(save_path,'rb')
b_new_test=pickle.load(f)
#print(b_new_test.shape)
f.close()

'''
Import UBM model
'''

ubm_dataset=loadmat("/Users/Mata/Documents/2017/学习/ws2017:18/PUL/forStudents/ubm/UBM_GMMNaive_MFCC_Spectrum0to8000Hz",mat_dtype=True)
#ubm_dataset=loadmat("C:\\Users\\hasee\\workspace\\workspace\\lab\\patRecDat\\forStudents\\ubm\\UBM_GMMNaive_MFCC_Spectrum0to8000Hz.mat",mat_dtype=True)
ubm_means=ubm_dataset['means']
ubm_var = ubm_dataset['var']
ubm_weights = ubm_dataset['weights'].ravel()
ubm_var_set=[]
K_value=49
gamma_UBM=1

#transfer variance of UBM to cov
for k in range(K_value):
    ubm_var_set.append(np.diag(ubm_var[k]))
ubm_var_set=np.array(ubm_var_set)
ubm_var=ubm_var_set

num_people=len(features.keys())

'''
Start Crossvalidation
'''
#process_bar=ShowProcess(10)
detection_rate_set=[]
start=time.time()
error_set={}  
num_samples=len(features.keys())
name_set=list(features.keys())
name_set.append("unknown")
correct_sum=0
false_sum=0
for cross_num in range(1):
    #process_bar.show_process()
    train_file_set=[]
    test_file_set=[]
    correct_num=0
    false_num=0
    '''
    Split the test and train set
    '''
    for name in features.keys():
        whole_set=features.get(name,'no such file').copy()
        test_file=whole_set[cross_num]
        train_num=list(range(10))
        train_num.remove(cross_num)
        test_file_set.append(test_file)
        #name_set.append(name)
        train_set=[]
        for num in train_num:
            train_set.append(whole_set[num])
        train_set=np.concatenate(train_set,axis=1)
        train_file_set.append(train_set)
        
    test_file_set.append(b_new_test)
    
    '''
    Start modeling and identification
    '''
    print("crossvalidation "+str(cross_num+1)+" start")
    process_bar_2=ShowProcess(num_samples)
    scores_set=np.zeros((num_samples+1,num_samples+1))
    for index_2 in range(num_samples):
        process_bar_2.show_process()
        b_train=train_file_set[index_2]
        gmm=GaussianMixture(n_components=K_value,covariance_type='full',max_iter=1,weights_init=ubm_weights,\
                            means_init=ubm_means,precisions_init=np.linalg.inv(ubm_var))
        gmm.fit(b_train.T)
        for index_1 in range(num_samples+1):
            #print("now test set "+str(index_1)+" is testing "+str(index_2))
            b_test=np.array(test_file_set[index_1])
            scores_set[index_1,index_2]=gmm.score(b_test.T)
    '''
    Enhancement : added the unknown detection part
    '''
    for index in range(num_samples+1):
        b_test=np.array(test_file_set[index])
        T_value=b_test.shape[1]
        unknown_score=Speaker_identification(b_test,ubm_means,ubm_var,ubm_weights,T_value)
        #scores_set[index,num_samples]=unknown_score
        scores_set[index,num_samples]=(1/T_value)*unknown_score
        
        
    '''
    Calculate the detection rate
    '''
    for index in range(num_samples+1):
        test_index=np.int(np.argwhere(scores_set[index,:]==max(scores_set[index,:])))
        if index == test_index:
            correct_num +=1
            correct_sum +=1
        else:
            false_num +=1
            false_sum +=1
            print('')
            print("error ! True: "+name_set[index]+" False: "+name_set[test_index])
        #print("time cost %5.1f second"%((time.time()-start)/60))

    process_bar_2.close()

    detection_rate=correct_num/(correct_num+false_num)
    print("crossvalidation "+str(cross_num+1)+" compeleted")
    print("cost time %5.1f minute"%((time.time()-start)/60))
    detection_rate=correct_num/(false_num+correct_num)
    detection_rate_set.append(detection_rate)
    print("the crossval "+str(cross_num+1)+" detection_rate is "+str(detection_rate))

print("the total detection rate is ",correct_sum/(correct_sum+false_sum))



crossvalidation 1 start


/Users/Mata/anaconda3/lib/python3.6/site-packages/sklearn/mixture/base.py:237: ConvergenceWarning: Initialization 1 did not converge. Try different init parameters, or increase max_iter, tol or check for degenerate data.
  % (init + 1), ConvergenceWarning)


KeyboardInterrupt: 

In [ ]:
scores_set[168,:]

In [7]:

from sklearn.mixture import GaussianMixture
import seaborn as sn
import pandas as pd
import matplotlib.pyplot as plt
#default

import pickle
import numpy as np
import time

#loadmat
from process_bar import ShowProcess
from scipy.io import loadmat,savemat

'''
Import features
'''

save_path="D:\\LAB\\lab\\task_2_version_4\\features.txt"
#save_path="/Users/Mata/Documents/lab/task_2_version_4/features.txt"
f = open(save_path,'rb')
features=pickle.load(f)
f.close()

save_path="D:\\LAB\\lab\\task_2_version_4\\unknown_features.txt"
#save_path="/Users/Mata/Documents/lab/task_2_version_4/unknown_features.txt"
f = open(save_path,'rb')
unknown_features=pickle.load(f)  #190 unregisterd user
#print(b_new_test.shape)
f.close()

'''
Import UBM model
'''

#ubm_dataset=loadmat("/Users/Mata/Documents/2017/学习/ws2017:18/PUL/forStudents/ubm/UBM_GMMNaive_MFCC_Spectrum0to8000Hz",mat_dtype=True)
ubm_dataset=loadmat("C:\\Users\\hasee\\workspace\\workspace\\lab\\patRecDat\\forStudents\\ubm\\UBM_GMMNaive_MFCC_Spectrum0to8000Hz.mat",mat_dtype=True)
ubm_means=ubm_dataset['means']
ubm_var = ubm_dataset['var']
ubm_weights = ubm_dataset['weights'].ravel()
ubm_var_set=[]
K_value=49
gamma_UBM=1

#transfer variance of UBM to cov
for k in range(K_value):
    ubm_var_set.append(np.diag(ubm_var[k]))
ubm_var_set=np.array(ubm_var_set)
ubm_var=ubm_var_set

num_people=len(features.keys())

'''
Start Crossvalidation
'''
#process_bar=ShowProcess(10)
detection_rate_set=[]
start=time.time()
error_set={}  
num_registered=len(features.keys())
num_unregistered=len(unknown_features.keys())
name_set=list(features.keys())
#name_set.append("unknown")
correct_sum=0
false_sum=0
ml_error_sum=0
fr_error_sum=0
fa_error_sum=0
threshold=-34.8
for cross_num in range(1):
    #process_bar.show_process()
    train_file_set=[]
    test_file_set=[]
    correct_num=0
    false_num=0
    ml_error=0
    fr_error=0
    fa_error=0
    '''
    Split the test and train set
    '''
    for name in features.keys():
        whole_set=features.get(name,'no such file').copy()
        test_file=whole_set[cross_num]
        train_num=list(range(10))
        train_num.remove(cross_num)
        test_file_set.append(test_file)
        #name_set.append(name)
        train_set=[]
        for num in train_num:
            train_set.append(whole_set[num])
        train_set=np.concatenate(train_set,axis=1)
        train_file_set.append(train_set)
        
    for name in unknown_features.keys():
        whole_set=unknown_features.get(name,'no such file').copy()
        test_file=whole_set[cross_num]
        test_file_set.append(test_file)

    
    '''
    Start modeling and identification
    '''
    print("crossvalidation "+str(cross_num+1)+" start")
    process_bar_2=ShowProcess(num_registered)
    scores_set=np.zeros((num_registered+num_unregistered,num_registered))
    for index_2 in range(num_registered):
        process_bar_2.show_process()
        b_train=train_file_set[index_2]
        gmm=GaussianMixture(n_components=K_value,covariance_type='full',max_iter=1,weights_init=ubm_weights,\
                            means_init=ubm_means,precisions_init=np.linalg.inv(ubm_var))
        gmm.fit(b_train.T)
        for index_1 in range(num_registered+num_unregistered):
            #print("now test set "+str(index_1)+" is testing "+str(index_2))
            b_test=np.array(test_file_set[index_1])
            scores_set[index_1,index_2]=gmm.score(b_test.T)

        
    '''
    Calculate the detection rate and error rate
    '''
    for index in range(num_registered+num_unregistered):
        if index < num_registered :
            if max(scores_set[index,:])>threshold*ubm:
                test_index=np.int(np.argwhere(scores_set[index,:]==max(scores_set[index,:])))
                if index == test_index:
                    correct_num +=1
                    correct_sum +=1
                else:
                    false_num +=1
                    false_sum +=1
                    ml_error +=1
                    ml_error_sum +=1
                    print('')
                    print("error ! True: "+name_set[index]+" False: "+name_set[test_index])
            else:
                    false_num+=1
                    false_sum+=1
                    fr_error+=1
                    fr_error_sum+=1
                    print('')
                    print("error ! True: "+name_set[index]+" False: unknowns")
        else:
            if max(scores_set[index,:])<=threshold:
                correct_num +=1
                correct_sum +=1
            else:
                false_num+=1
                false_sum+=1
                fa_error+=1
                fa_error_sum+=1
                print('')
                print("error ! Unknown misidentified")
                
            
        #print("time cost %5.1f second"%((time.time()-start)/60))

    process_bar_2.close()

    detection_rate=correct_num/(correct_num+false_num)
    print("crossvalidation "+str(cross_num+1)+" compeleted")
    print("cost time %5.1f minute"%((time.time()-start)/60))
    detection_rate=correct_num/(false_num+correct_num)
    ml_rate=100*ml_error/(num_registered)
    fr_rate=100*fr_error/(num_registered)
    fa_rate=100*fa_error/(num_unregistered)
    aer_rate=100*(ml_error+fr_error+fa_error)/(num_registered+num_unregistered)
    #detection_rate_set.append(detection_rate)
    print("the crossval "+str(cross_num+1)+" detection_rate is "+str(detection_rate))
    print("ML %5.1f, FR, %5.1f , FA, %5.1f, AER, %5.1f"%(ml_rate,fr_rate,fa_rate,aer_rate))
print("the total detection rate is ",correct_sum/(correct_sum+false_sum))
print("the total detection rate is ",correct_sum/(correct_sum+false_sum))



crossvalidation 1 start


D:\ProgramData\Anaconda3\lib\site-packages\sklearn\mixture\base.py:237: ConvergenceWarning: Initialization 1 did not converged. Try different init parameters, or increase max_iter, tol or check for degenerate data.
  % (init + 1), ConvergenceWarning)


[>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>]100.00%
error ! Unknown misidentified

error ! Unknown misidentified

error ! Unknown misidentified

error ! Unknown misidentified

error ! Unknown misidentified

error ! Unknown misidentified

error ! Unknown misidentified

error ! Unknown misidentified

error ! Unknown misidentified

error ! Unknown misidentified

error ! Unknown misidentified

error ! Unknown misidentified

error ! Unknown misidentified

error ! Unknown misidentified

error ! Unknown misidentified

error ! Unknown misidentified

error ! Unknown misidentified

error ! Unknown misidentified

error ! Unknown misidentified

error ! Unknown misidentified

error ! Unknown misidentified

error ! Unknown misidentified

error ! Unknown misidentified

error ! Unknown misidentified

error ! Unknown misidentified

error ! Unknown misidentified

error ! Unknown misidentified

error ! Unknown misidentified

error ! Unknown misidentified

error ! Unknown misidentified

error ! Un

In [2]:
from Speaker_identification import Speaker_identification
from density_func import density_func, naive_G_U
from sklearn.mixture import GaussianMixture
import seaborn as sn
import pandas as pd
import matplotlib.pyplot as plt
#default

import pickle
import numpy as np
import time

#loadmat
from process_bar import ShowProcess
from scipy.io import loadmat,savemat

'''
Import features
'''

#save_path="D:\\LAB\\lab\\task_2_version_4\\features.txt"
save_path="/Users/Mata/Documents/lab/task_2_version_4/features.txt"
f = open(save_path,'rb')
features=pickle.load(f)
f.close()

#save_path="D:\\LAB\\lab\\task_2_version_4\\unknown_features.txt"
save_path="/Users/Mata/Documents/lab/task_2_version_4/unknown_features.txt"
f = open(save_path,'rb')
unknown_features=pickle.load(f)  #190 unregisterd user
#print(b_new_test.shape)
f.close()

'''
Import UBM model
'''

ubm_dataset=loadmat("/Users/Mata/Documents/2017/学习/ws2017:18/PUL/forStudents/ubm/UBM_GMMNaive_MFCC_Spectrum0to8000Hz",mat_dtype=True)
#ubm_dataset=loadmat("C:\\Users\\hasee\\workspace\\workspace\\lab\\patRecDat\\forStudents\\ubm\\UBM_GMMNaive_MFCC_Spectrum0to8000Hz.mat",mat_dtype=True)
ubm_means=ubm_dataset['means']
ubm_var = ubm_dataset['var']
ubm_weights = ubm_dataset['weights'].ravel()
ubm_var_set=[]
K_value=49
gamma_UBM=1

#transfer variance of UBM to cov
for k in range(K_value):
    ubm_var_set.append(np.diag(ubm_var[k]))
ubm_var_set=np.array(ubm_var_set)
ubm_var=ubm_var_set

num_people=len(features.keys())

'''
Start Crossvalidation
'''
#process_bar=ShowProcess(10)
detection_rate_set=[]
ml_rate_set=np.zeros((10,41))
fr_rate_set=np.zeros((10,41))
fa_rate_set=np.zeros((10,41))
aer_rate_set=np.zeros((10,41))
start=time.time()
error_set={}  
num_registered=len(features.keys())
num_unregistered=len(unknown_features.keys())
name_set=list(features.keys())
#name_set.append("unknown")
for cross_num in range(10):
    print("crossvalidation "+str(cross_num)+" start")
    #process_bar.show_process()
    train_file_set=[]
    test_file_set=[]
    '''
    Split the test and train set
    '''
    for name in features.keys():
        whole_set=features.get(name,'no such file').copy()
        test_file=whole_set[cross_num]
        train_num=list(range(10))
        train_num.remove(cross_num)
        test_file_set.append(test_file)
        #name_set.append(name)
        train_set=[]
        for num in train_num:
            train_set.append(whole_set[num])
        train_set=np.concatenate(train_set,axis=1)
        train_file_set.append(train_set)
        
    for name in unknown_features.keys():
        whole_set=unknown_features.get(name,'no such file').copy()
        test_file=whole_set[cross_num]
        test_file_set.append(test_file)

    
        '''
    Enhancement : added the unknown detection part
    '''
    ubm_scores=[]
    print("ubm model start")
    process_bar_1=ShowProcess(num_registered+num_unregistered)
    for index in range(num_registered+num_unregistered):
        process_bar_1.show_process()
        b_test=np.array(test_file_set[index])
        T_value=b_test.shape[1]
        unknown_score=Speaker_identification(b_test,ubm_means,ubm_var,ubm_weights,T_value)
        #scores_set[index,num_samples]=unknown_score
        ubm_scores.append((1/T_value)*unknown_score)
    process_bar_1.close()
    '''
    Start modeling and identification
    '''
    #print("crossvalidation "+str(cross_num+1)+" start")
    process_bar_2=ShowProcess(num_registered)
    scores_set=np.zeros((num_registered+num_unregistered,num_registered))
    for index_2 in range(num_registered):
        process_bar_2.show_process()
        b_train=train_file_set[index_2]
        gmm=GaussianMixture(n_components=K_value,covariance_type='full',max_iter=1,weights_init=ubm_weights,\
                            means_init=ubm_means,precisions_init=np.linalg.inv(ubm_var))
        gmm.fit(b_train.T)
        for index_1 in range(num_registered+num_unregistered):
            #print("now test set "+str(index_1)+" is testing "+str(index_2))
            b_test=np.array(test_file_set[index_1])
            scores_set[index_1,index_2]=gmm.score(b_test.T)

        
    '''
    Calculate the detection rate and error rate
    '''
    for index_1,threshold in enumerate([0.3,0.31,0.32,0.33,0.34,0.35,0.36,0.37,0.38,0.39,0.40,\
                     0.41,0.42,0.43,0.44,0.45,0.46,0.47,0.48,0.49,0.50,\
                     0.51,0.52,0.53,0.54,0.55,0.56,0.57,0.58,0.59,0.60,\
                     0.61,0.62,0.63,0.64,0.65,0.66,0.67,0.68,0.69,0.70]):
    #process_bar.show_process()
        correct_num=0
        false_num=0
        ml_error=0
        fr_error=0
        fa_error=0
        for index in range(num_registered+num_unregistered):
            if index < num_registered :
                if max(scores_set[index,:])>threshold*ubm_scores[index]:
                    test_index=np.int(np.argwhere(scores_set[index,:]==max(scores_set[index,:])))
                    if index == test_index:
                        correct_num +=1
                    else:
                        false_num +=1
                        ml_error +=1
                        #print('')
                        #print("error ! True: "+name_set[index]+" False: "+name_set[test_index])
                else:
                        false_num+=1
                        fr_error+=1
                        #print('')
                        #print("error ! True: "+name_set[index]+" False: unknowns")
            else:
                if max(scores_set[index,:])<=threshold*ubm_scores[index]:
                    correct_num +=1
                else:
                    false_num+=1
                    fa_error+=1
                    #print('')
                    #print("error ! Unknown misidentified")
        ml_rate=100*ml_error/(num_registered)
        fr_rate=100*fr_error/(num_registered)
        fa_rate=100*fa_error/(num_unregistered)
        aer_rate=100*(ml_error+fr_error+fa_error)/(num_registered+num_unregistered)
        ml_rate_set[cross_num,index_1]=ml_rate
        fr_rate_set[cross_num,index_1]=fr_rate
        fa_rate_set[cross_num,index_1]=fa_rate
        aer_rate_set[cross_num,index_1]=aer_rate
        print("threshold "+str(threshold)+" compeleted")
        #print("time cost %5.1f second"%((time.time()-start)/60))
        print("ML %5.1f, FR, %5.1f , FA, %5.1f, AER, %5.1f"%(ml_rate,fr_rate,fa_rate,aer_rate))
    process_bar_2.close()

    detection_rate=correct_num/(correct_num+false_num)
    print("cross_validation "+str(cross_num)+" compeleted")
    print("cost time %5.1f minute"%((time.time()-start)/60))
    detection_rate=correct_num/(false_num+correct_num)
    #detection_rate_set.append(detection_rate)
    print("cross validation "+str(cross_num)+" detection_rate is "+str(detection_rate))
#print("the total detection rate is ",correct_sum/(correct_sum+false_sum))

#save the error rate 
error_rate_set={}
error_rate_set.setdefault('ml')
error_rate_set.setdefault('fr')
error_rate_set.setdefault('fa')
error_rate_set.setdefault('aer')
error_rate_set['ml']=ml_rate_set
error_rate_set['fr']=fr_rate_set
error_rate_set['fa']=fa_rate_set
error_rate_set['aer']=aer_rate_set
save_path="/Users/Mata/Documents/lab/task_2_version_4/error_rate_set.txt"
f = open(save_path,'wb')
pickle.dump(error_rate_set,f)
f.close()

crossvalidation 0 start
ubm model start
done>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>]100.00%


/Users/Mata/anaconda3/lib/python3.6/site-packages/sklearn/mixture/base.py:237: ConvergenceWarning: Initialization 1 did not converge. Try different init parameters, or increase max_iter, tol or check for degenerate data.
  % (init + 1), ConvergenceWarning)


threshold 0.3 compeleted>>>>>>>>>>>>>>>>>>>>>>>>>>>]100.00%
ML   0.0, FR, 100.0 , FA,   0.0, AER,  46.9
threshold 0.31 compeleted
ML   0.0, FR, 100.0 , FA,   0.0, AER,  46.9
threshold 0.32 compeleted
ML   0.0, FR, 100.0 , FA,   0.0, AER,  46.9
threshold 0.33 compeleted
ML   0.0, FR, 100.0 , FA,   0.0, AER,  46.9
threshold 0.34 compeleted
ML   0.0, FR, 100.0 , FA,   0.0, AER,  46.9
threshold 0.35 compeleted
ML   0.0, FR, 100.0 , FA,   0.0, AER,  46.9
threshold 0.36 compeleted
ML   0.0, FR, 100.0 , FA,   0.0, AER,  46.9
threshold 0.37 compeleted
ML   0.0, FR, 100.0 , FA,   0.0, AER,  46.9
threshold 0.38 compeleted
ML   0.0, FR, 100.0 , FA,   0.0, AER,  46.9
threshold 0.39 compeleted
ML   0.0, FR, 100.0 , FA,   0.0, AER,  46.9
threshold 0.4 compeleted
ML   0.0, FR, 100.0 , FA,   0.0, AER,  46.9
threshold 0.41 compeleted
ML   0.0, FR, 100.0 , FA,   0.0, AER,  46.9
threshold 0.42 compeleted
ML   0.0, FR,  98.2 , FA,   0.5, AER,  46.4
threshold 0.43 compeleted
ML   0.0, FR,  97.0 , FA,   1.1

done>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>]100.00%
threshold 0.3 compeleted>>>>>>>>>>>>>>>>>>>>>>>>>>>]100.00%
ML   0.0, FR, 100.0 , FA,   0.0, AER,  46.9
threshold 0.31 compeleted
ML   0.0, FR, 100.0 , FA,   0.0, AER,  46.9
threshold 0.32 compeleted
ML   0.0, FR, 100.0 , FA,   0.0, AER,  46.9
threshold 0.33 compeleted
ML   0.0, FR, 100.0 , FA,   0.0, AER,  46.9
threshold 0.34 compeleted
ML   0.0, FR, 100.0 , FA,   0.0, AER,  46.9
threshold 0.35 compeleted
ML   0.0, FR, 100.0 , FA,   0.0, AER,  46.9
threshold 0.36 compeleted
ML   0.0, FR, 100.0 , FA,   0.0, AER,  46.9
threshold 0.37 compeleted
ML   0.0, FR, 100.0 , FA,   0.0, AER,  46.9
threshold 0.38 compeleted
ML   0.0, FR,  99.4 , FA,   0.0, AER,  46.6
threshold 0.39 compeleted
ML   0.0, FR,  99.4 , FA,   0.0, AER,  46.6
threshold 0.4 compeleted
ML   0.0, FR,  99.4 , FA,   0.0, AER,  46.6
threshold 0.41 compeleted
ML   0.0, FR,  99.4 , FA,   0.0, AER,  46.6
threshold 0.42 compeleted
ML   0.0, FR,  97.6 , FA,   0.5, AER,  46

threshold 0.59 compeleted
ML   0.6, FR,  35.1 , FA,  60.0, AER,  48.6
threshold 0.6 compeleted
ML   0.6, FR,  28.6 , FA,  66.8, AER,  49.2
threshold 0.61 compeleted
ML   0.6, FR,  23.8 , FA,  72.6, AER,  50.0
threshold 0.62 compeleted
ML   0.6, FR,  21.4 , FA,  75.8, AER,  50.6
threshold 0.63 compeleted
ML   0.6, FR,  16.7 , FA,  76.3, AER,  48.6
threshold 0.64 compeleted
ML   0.6, FR,  14.9 , FA,  77.9, AER,  48.6
threshold 0.65 compeleted
ML   0.6, FR,  12.5 , FA,  82.1, AER,  49.7
threshold 0.66 compeleted
ML   0.6, FR,  10.7 , FA,  85.3, AER,  50.6
threshold 0.67 compeleted
ML   0.6, FR,   9.5 , FA,  87.9, AER,  51.4
threshold 0.68 compeleted
ML   0.6, FR,   8.3 , FA,  88.9, AER,  51.4
threshold 0.69 compeleted
ML   0.6, FR,   7.1 , FA,  89.5, AER,  51.1
threshold 0.7 compeleted
ML   0.6, FR,   5.4 , FA,  90.0, AER,  50.6
done
cross_validation 5 compeleted
cost time  51.5 minute
cross validation 5 detection_rate is 0.49441340782122906
crossvalidation 6 start
ubm model start
done>>>

done>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>]100.00%
threshold 0.3 compeleted>>>>>>>>>>>>>>>>>>>>>>>>>>>]100.00%
ML   0.0, FR, 100.0 , FA,   0.0, AER,  46.9
threshold 0.31 compeleted
ML   0.0, FR, 100.0 , FA,   0.0, AER,  46.9
threshold 0.32 compeleted
ML   0.0, FR, 100.0 , FA,   0.0, AER,  46.9
threshold 0.33 compeleted
ML   0.0, FR, 100.0 , FA,   0.0, AER,  46.9
threshold 0.34 compeleted
ML   0.0, FR, 100.0 , FA,   0.0, AER,  46.9
threshold 0.35 compeleted
ML   0.0, FR, 100.0 , FA,   0.0, AER,  46.9
threshold 0.36 compeleted
ML   0.0, FR, 100.0 , FA,   0.0, AER,  46.9
threshold 0.37 compeleted
ML   0.0, FR,  99.4 , FA,   0.0, AER,  46.6
threshold 0.38 compeleted
ML   0.0, FR,  99.4 , FA,   0.0, AER,  46.6
threshold 0.39 compeleted
ML   0.0, FR,  99.4 , FA,   0.5, AER,  46.9
threshold 0.4 compeleted
ML   0.0, FR,  98.2 , FA,   1.1, AER,  46.6
threshold 0.41 compeleted
ML   0.0, FR,  97.0 , FA,   1.1, AER,  46.1
threshold 0.42 compeleted
ML   0.0, FR,  97.0 , FA,   1.1, AER,  46

-34.5477372316


In [29]:
maxscores_set[167,:]

array([-35.16739576, -35.07816931, -38.01358023, -34.55097793,
       -35.14696366, -35.6207894 , -34.7130317 , -35.01925468,
       -34.33254802, -36.27045808, -34.61785226, -35.49230568,
       -37.69402409, -34.25104765, -36.31212361, -35.97375522,
       -35.50952335, -35.15286143, -33.74885409, -34.49444825,
       -35.35892987, -34.60846175, -34.97539477, -35.13104644,
       -36.96582448, -34.91243364, -35.47891614, -33.80781995,
       -36.25763411, -34.23478348, -35.06876325, -35.76196391,
       -35.07293055, -34.67811964, -34.10971356, -36.09592121,
       -34.68701369, -36.39633513, -36.18863364, -34.87000502,
       -36.24206897, -36.7490916 , -35.00100171, -33.58947464,
       -35.06855828, -33.68204238, -34.52316123, -34.99981336,
       -34.15786625, -33.24105488, -34.12014232, -33.38512227,
       -34.57652049, -36.21805711, -35.75220225, -35.17655419,
       -35.31555152, -34.64501274, -34.64348095, -35.91898606,
       -34.30680853, -33.87006891, -35.15421479, -35.60